# Scraping xml from bgg because the age of the data is bothering me

In [1]:
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs
import pandas as pd
from random import randint
import os
import csv
import pickle
from tqdm import tqdm

In [2]:
games = []
categories = set()
mechanics = set()

# Functions

In [3]:
def get_game_data(game_id, retry_limit=3):
    response = requests.get(f'https://www.boardgamegeek.com/xmlapi2/thing?id={game_id}&stats=1')
    soup = BeautifulSoup(response.content, 'xml')

    # If it returns None, try 3 more times and then otherwise return None
    if soup.find('item') is None:
        print(f"Unexpected response for game ID {game_id}: HTTP {response.status_code}, content: {response.content}")
        if retry_limit > 0:
            time.sleep(1)
            return get_game_data(game_id, retry_limit=retry_limit -1)
        else:
            return None

    game = {}

    game['rank'] = None  # This information is not available in XML API, placeholder for future usage.
    game['bgg_url'] = f'https://boardgamegeek.com/boardgame/{game_id}'
    game['game_id'] = game_id
    game['name'] = soup.find('name').get('value')
    
    game['min_players'] = soup.find('minplayers').get('value')
    game['max_players'] = soup.find('maxplayers').get('value')
    game['avg_time'] = soup.find('playingtime').get('value')
    game['min_time'] = soup.find('minplaytime').get('value')
    game['max_time'] = soup.find('maxplaytime').get('value')
    game['weight'] = soup.find('averageweight').get('value')
    
    game['year'] = soup.find('yearpublished').get('value')
    game['age'] = soup.find('minage').get('value')

    game['avg_rating'] = soup.find('average').get('value')
    game['geek_rating'] = soup.find('bayesaverage').get('value')
    game['num_votes'] = soup.find('usersrated').get('value')
    game['owned'] = soup.find('owned').get('value')
    game['designer'] = ', '.join(d.get('value') for d in soup.find_all('link', type='boardgamedesigner'))

    game['categories'] = [c.get('value') for c in soup.find_all('link', type='boardgamecategory')]
    categories.update(game['categories'])

    game['mechanics'] = [m.get('value') for m in soup.find_all('link', type='boardgamemechanic')]
    mechanics.update(game['mechanics'])

    return game
    

In [4]:
def get_id_from_page(game_page_url):
    response = requests.get(game_page_url)
    if response.statuscod !=200:
        return None
    
    url = urlparse(response.url)
    game_id = parse_qs(url.query).get('id', None)
    
    if game_id is not None:
        return game_id[0]

In [5]:
def get_top_game_ids(page_num):
    response = requests.get(f'https://boardgamegeek.com/browse/boardgame/page/{page_num}')
    soup = BeautifulSoup(response.content, 'html.parser')
    game_ids = []
    
    if page_num >= 21:
        links = soup.select('td.collection_thumbnail a[href^="/boardgame/"]')
        for link in links:
            game_url = 'https://boardgamegeek.com' + link['href']
            game_id = get_game_id_from_page(game_url)
            if game_id is not None:
                game_ids.append(game_id)
    
    # Find all the boardgame links and extract the IDs
    links = soup.select('td.collection_thumbnail a[href^="/boardgame/"]')
    game_ids = [link['href'].split('/')[2] for link in links]
    
    
    return game_ids

In [6]:
def pickle_here(games_list, rank):
    with open(f'data/pickle/games_{rank}.pickle', 'wb') as f:
        pickle.dump(games_list, f)

In [7]:
def get_links_from_page(page_num):
    response = requests.get(f'https://boardgamegeek.com/browse/boardgame/page/{page_num}')
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all the links on the page
    links = soup.select('a[href]')
    link_urls = [link['href'] for link in links]
    
    return link_urls

## Implementation

## Creating csv and writing header

In [7]:
with open('data/basic_data_2023.csv', 'w', newline='', encoding='utf-8') as csvfile:
    
    fields = [
    'rank',
    'bgg_url',
    'game_id',
    'name',
    'min_players',
    'max_players',
    'avg_time',
    'min_time',
    'max_time',
    'weight',
    'year',
    'age',
    'avg_rating',
    'geek_rating',
    'num_votes',
    'owned',
    'designer',
    'categories',
    'mechanics',
    ]
    
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()

### Scraping website

In [8]:
# get top game ids
top_game_ids = []

with tqdm(total=21) as pbar:
    for page_num in range(1, 22):
        try:
            top_game_ids.extend(get_top_game_ids(page_num))
            time.sleep(1)
            pbar.update()
        except Exception as e:
            tqdm.write(f'Error {e} occurred on page number {page_num}')

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:25<00:00,  1.20s/it]


In [9]:
# top_games_pg = get_top_game_ids(21)
# top_games_pg

In [11]:
games = []

#open csv and scrape website
with open('data/basic_data_2023.csv', 'a', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    with tqdm(total=len(top_game_ids)) as pbar:
        for rank, game_id in enumerate(top_game_ids, start=1):
            try:
                game = get_game_data(game_id)
                if game is not None:
                    game['rank'] = rank
                    games.append(game)
                    pbar.set_description(f'Successfully fetched game ID: {game_id}, Rank: {rank}')
                    writer.writerow(game)
                    
                    if rank % 500==0:
                        pickle_here(games, rank)
                    
                else:
                    pbar.set_description(f'No data discovered for game ID: {game_id}')
                pbar.update()
            except Exception as e:
                tqdm.write(f'Error {e} occurred fetching game ID: {game_id}')
            time.sleep(randint(1,3))

Successfully fetched game ID: 174785, Rank: 635:  32%|█████████▌                    | 635/2000 [23:15<48:43,  2.14s/it]

Unexpected response for game ID 37904: HTTP 429, content: b'<error>\n    <message>Rate limit exceeded.</message>\n</error>'


Successfully fetched game ID: 357563, Rank: 651:  33%|█████████▊                    | 651/2000 [23:53<52:40,  2.34s/it]

Unexpected response for game ID 296912: HTTP 429, content: b'<error>\n    <message>Rate limit exceeded.</message>\n</error>'


Successfully fetched game ID: 58281, Rank: 663:  33%|██████████▎                    | 663/2000 [24:21<59:03,  2.65s/it]

...

Successfully fetched game ID: 204574, Rank: 1993: 100%|█████████████████████████▉| 1993/2000 [1:19:25<00:17,  2.47s/it]

Unexpected response for game ID 12589: HTTP 429, content: b'<error>\n    <message>Rate limit exceeded.</message>\n</error>'


Successfully fetched game ID: 189660, Rank: 1999: 100%|█████████████████████████▉| 1999/2000 [1:19:41<00:02,  2.55s/it]

Unexpected response for game ID 19526: HTTP 429, content: b'<error>\n    <message>Rate limit exceeded.</message>\n</error>'


Successfully fetched game ID: 19526, Rank: 2000: 100%|███████████████████████████| 2000/2000 [1:19:47<00:00,  2.39s/it]
